# Manipulating databases

* Produce PQ db extension to GAP-18
* initiate active learning from GAP

In [181]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [182]:
from ase.io.extxyz import read_xyz
import numpy as np
from ase.atoms import Atoms
from ase.io.formats import index2range
from ase.io.cfg import read_cfg
from io import StringIO, UnsupportedOperation
from mtp import *
import os
from matplotlib import pyplot as plt
from quippy.potential import Potential
import pickle
from Ge_analysis import *
from Ge_calculation import *
import matplotlib.pyplot as plt
from matscipy.rings import ring_statistics
from datetime import datetime
import pymatgen.ext.matproj as mp
import pymatgen.io.ase as pase
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from numpy.random import normal
from ase.io.extxyz import write_xyz
from ase.io.lammpsdata import write_lammps_data
from os.path import join

In [183]:
mpl.style.reload_library()
mpl.style.use('VLD')

# Labelling

worry about configurations that didn't have a virial to begin with (i.e. isolated atoms) in g18_db

In [184]:
GAP_18_dir = '/u/vld/hert5155/jup_remote/Ge_od/Si_GAP_training/libAtoms-silicon-testing-framework-fc252cb/models/GAP'
os.chdir('/u/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP')
GAP_18_pot = Potential(param_filename=GAP_18_dir+'/gp_iter6_sparse9k.xml')

/u/vld/hert5155/ase/ase/calculators/calculator.py:507: FutureWarning: The keyword "ignore_bad_restart_file" is deprecated and will be removed in a future version of ASE.  Passing more than one positional argument to Calculator is also deprecated and will stop functioning in the future.  Please pass arguments by keyword (key=value) except optionally the "restart" keyword.
  warnings.warn(FutureWarning(


In [185]:
with open(GAP_18_dir + '/gp_iter6_sparse9k.xml.xyz', 'r') as f:
    GAP_18_db = list(read_xyz(f, index=slice(0,None)))

In [186]:
with open('/home/joe/Documents/Ge_od/Potentials/Si_myDB_reduction_testing/Si_myDB_64_125_216_train.xyz', 'r') as f:
    myDB = list(read_xyz(f, index=slice(0,None)))

FileNotFoundError: [Errno 2] No such file or directory: '/home/joe/Documents/Ge_od/Potentials/Si_myDB_reduction_testing/Si_myDB_64_125_216_train.xyz'

In [ ]:
db = myDB
# db = GAP_18_db
for i, val in enumerate(db):

    GAP_18_pot.calculate(val, properties=['energies', 'forces', 'stress'])
    val.info['gap18_energy'] = GAP_18_pot.results['energy']
    val.info['gap18_virial'] = GAP_18_pot.results['virial']
    val.arrays['gap18_force'] = GAP_18_pot.results['forces']

In [ ]:
GAP_18_pot.calculate(GAP_18_db[0], properties=['energies', 'forces', 'stress'])
GAP_18_pot.results['virial']

In [99]:
mtp_stress_test = MTP('/u/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP/MTPs/MTPs/u16_m1.5_fw0.1_GAP18db.mtp',
                     mtp_command='/u/vld/hert5155/mlip-2/bin/mlp')

# Load up the labelled GAP_18 db

In [ ]:
# with open('GAP_18_db_GAPlabelled_MTP4.pickle', 'wb') as f:
#     pickle.dump(GAP_18_db, f)

In [188]:
with open('GAP_18_db_GAPlabelled.pickle', 'rb') as f:
    GAP_18_db = pickle.load(f)

define both pos and negstresses in the g18 db

In [210]:
for ct, i in enumerate(GAP_18_db):
    try:
        i.info['gap18_stress'] = -i.info['stress']
        i.info['gap18_posstress'] = i.info['stress']
    except:
        print(ct)

1733
1755
1777
2146


In [ ]:
# with open('mydb_GAPlabelled_MTP4.pickle', 'wb') as f:
#     pickle.dump(myDB, f)

In [ ]:
with open('mydb_GAPlabelled_MTP4.pickle', 'rb') as f:
    myDB = pickle.load(f)

In [ ]:
# with open('GAP_18_db_GAPlabelled.cfg', 'w') as f:
#     write_cfg_db(f, GAP_18_db, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [ ]:
# with open('GAP_18_db_DFTlabelled.cfg', 'w') as f:
#     write_cfg_db(f, GAP_18_db, energy_name='dft_energy', force_name='dft_force', virial_name='dft_virial')

# Checking that the stress-fitting hasn't been a mistake

training u16s on g18 db, then compare stress calculations for size and magnitude consistency - especially the mtp-lammps implementation version, important for MD


In [118]:
negstress = deepcopy(GAP_18_db); posstress = deepcopy(GAP_18_db); virial = deepcopy(GAP_18_db)

In [119]:
negstress[1733].info#, negstress[150].info['gap18_virial']/i.get_volume() 

{'time': 2000.0,
 'i_step': 2000,
 'iprint': 2,
 'reuse': 'NULL',
 'task': 'SinglePoint',
 'calculate_stress': True,
 'data_distribution': 'GVECTOR',
 'opt_strategy': 'memory',
 'xc_functional': 'PW91',
 'cut_off_energy': '250.0000000 eV',
 'fine_grid_scale': 4.0,
 'finite_basis_corr': 'automatic',
 'spin_polarized': False,
 'elec_energy_tol': '0.0000001 ev',
 'max_scf_cycles': 250,
 'fix_occupancy': False,
 'smearing_width': '0.0500000 eV',
 'num_dump_cycles': 0,
 'mixing_scheme': 'pulay',
 'mix_history_length': 20,
 'kpoints_mp_grid': array([5, 5, 5]),
 'castep_file_name': 'interstitial_L1.xyz@21/filepot/filepot.castep',
 'config_type': 'interstitial',
 'dft_energy': -35392.520445,
 'cutoff': 6.0,
 'nneightol': 1.2,
 'gap18_energy': -35392.56708721385,
 'gap18_virial': array([[46.405085  ,  7.69286087,  3.41361857],
        [ 7.69286087, 48.00199955, 12.62122015],
        [ 3.41361857, 12.62122015, 43.2511007 ]])}

In [120]:
for ct, i in enumerate(negstress):
    try:
        i.info['stress'] *= -1
    except:
        i.calc = GAP_18_pot
        i.info['stress'] = -1*i.get_stress()
        print(ct)

1733
1755
1777
2146


In [123]:
for ct, i in enumerate(posstress):
    try:
        i.info['stress'] = i.info['stress']
    except:
        i.calc = GAP_18_pot
        i.info['stress'] = i.get_stress()
        print(ct)

1733
1755
1777
2146


In [122]:
with open('g18_db_negstres.cfg', 'w') as f:
    write_cfg_db(f, negstress, energy_name='gap18_energy', force_name='gap18_force', virial_name='stress')


In [124]:
with open('g18_db_posstress.cfg', 'w') as f:
    write_cfg_db(f, posstress, energy_name='gap18_energy', force_name='gap18_force', virial_name='stress')


In [125]:
with open('g18_db_virial.cfg', 'w') as f:
    write_cfg_db(f, virial, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

load up the erroneously trained MP-mtp

In [143]:
u16_mpmq = MTP('/home/tridymite/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP/MTPs/MTPs/u16_m1.5_mpmqGAP18db_GAPlabelled.mtp',
              parallel=False, mtp_command='/u/vld/hert5155/mlip-2/bin/mlp')
u16_mq = MTP('/home/tridymite/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP/MTPs/MTPs/u16_m1.5_mqGAP18db_GAPlabelled.mtp',
            parallel=False, mtp_command='/u/vld/hert5155/mlip-2/bin/mlp')

clearly the MP and mq-involved learning is a mistake:

In [145]:
db5[-1].calc = GAP_18_pot
print(db5[-1].get_stress())
db5[-1].calc = u16_mpmq
print(db5[-1].get_stress())
db5[-1].calc = u16_mq
print(db5[-1].get_stress())
db5[-1].info['gap18_virial']

[-0.12915563 -0.11438394 -0.11737569  0.0031333  -0.01035326  0.01171124]
[1975.79848 1617.92787 1377.9367    63.20511  -25.44105 -111.96074]
[1597.21428 1524.43094 1812.74613  -83.27763  126.22169 -140.77958]


array([-0.12915563, -0.11438394, -0.11737569,  0.0031333 , -0.01035326,
        0.01171124])

Need to sort out the sign-convention stuff now - appears that we should be using the negative of the stress (i.e. as VASP outputs)

In [154]:
GAP_18_db[100].calc = GAP_18_pot
GAP_18_db[100].info['gap18_virial'], -1*GAP_18_db[100].get_stress()

(array([[ 1.29953211, -0.00709474, -0.10932162],
        [-0.00709474,  1.13899967,  0.03959797],
        [-0.10932162,  0.03959797,  2.55389325]]),
 array([ 0.09818351,  0.08605481,  0.19295422,  0.00299174, -0.00825957,
        -0.00053603]))

In [159]:
for ct, i in enumerate(GAP_18_db):
    try:
        i.info['dft_stress'] = i.info['dft_virial']/i.get_volume() # defines poss-stress
    except:
        print(ct)

0
1221
1222
1223
1224
1225
1226
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
142

In [160]:
with open('/u/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP/stress_correct/g18_db_DFT-labelled.cfg', 'w') as f:
    write_cfg_db(f, GAP_18_db, energy_name='dft_energy', force_name='dft_force', virial_name='dft_stress')

## Writing all melt-quench data to DB for fitting


In [299]:
from re import findall

In [310]:
sb = '/u/vld/hert5155/jup_remote/Ge_od/Structure_databases'
mq_dirs = [os.path.join(sb, i) for i in ['run_64_atoms', 'run_125_atoms', 'run_216_atoms']]

mq_db = []

for i in mq_dirs:
    for j in os.listdir(i):
        if 'run_' in j and 'castep' not in j:
            for k in os.listdir(os.path.join(i, j, 'NPT', )):
                at = read_cfg(os.path.join(i,j,'NPT',k))
                at.info['rundir'] = j
                at.info['timestep'] = int(findall("\d+", k)[1])/1000
                mq_db.append(at)
l = len(mq_db)
# for i, val in enumerate(mq_db):
#
#     GAP_18_pot.calculate(val, properties=['energies', 'forces', 'stress'])
#     val.info['gap18_energy'] = GAP_18_pot.results['energy']
#     val.info['gap18_virial'] = GAP_18_pot.results['virial']
#     val.arrays['gap18_force'] = GAP_18_pot.results['forces']
#     if (i//l * 100) % 10 == 0:
#         print('{}% done'.format(i//l *100), datetime.now().time())

# with open('mq_db_GAP-18labelled.pickle', 'wb') as f:
#     pickle.dump(mq_db, f)

In [311]:
mq_db[50].info

{'rundir': 'run_1', 'timestep': 151.0}

In [274]:
with open('/u/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP/mqdb_GAP18labelled/mq_db_GAP-18labelled_3.pickle', 'rb') as f:
    mq_db_G18lab = pickle.load(f)


In [312]:
for i in range(len(mq_db_G18lab)):
    mq_db_G18lab[i].info['timestep'] = mq_db[i].info['timestep']

In [276]:
# need to modify the stresses before writing them (POST fiasco: or do we..)

for i in range(len(mq_db_G18lab)):
    v = -mq_db_G18lab[i].info['gap18_virial']*mq_db_G18lab[i].get_volume()
    mq_db_G18lab[i].info['gap18_virial'] = v

In [313]:
GAP_18_pot.calculate(mq_db_G18lab_train[0], properties=['stress'])
mq_db_G18lab_train[0].info, GAP_18_pot.extra_results

({'rundir': 'run_1',
  'gap18_energy': -10400.241841200821,
  'gap18_virial': array([[-4.43869229, -5.58105065,  0.30902159],
         [-5.58105065,  0.38781512,  1.22994423],
         [ 0.30902159,  1.22994423, -2.0113274 ]]),
  'timestep': 127.0},
 {'config': {'virial': array([[-4.43869228, -5.58105063,  0.30902156],
          [-5.58105063,  0.38781513,  1.22994423],
          [ 0.30902156,  1.22994423, -2.0113274 ]])},
  'atoms': {}})

In [314]:
# split into 80:20 test:training with entirely separate runs
rundirs = [int(i.info['rundir'][4:]) for i in mq_db_G18lab]
rundirs = np.array(rundirs)

In [315]:
# keep back run numbers 8-27
v_args = np.array([[]]); t_args = np.array([[]])
for i in range(8,28):
    v_args = np.concatenate((v_args, np.where(rundirs==i)), axis=1)

for i in range(8):
    t_args = np.concatenate((t_args, np.where(rundirs==i)), axis=1)
for i in range(28,101):
    t_args = np.concatenate((t_args, np.where(rundirs==i)), axis=1)

In [316]:
mq_db_G18lab_train = [mq_db_G18lab[int(i)] for i in t_args[0]]
mq_db_G18lab_val = [mq_db_G18lab[int(i)] for i in v_args[0]]

In [324]:
for i in mq_db_G18lab_val:
    i.info['config_type'] = 'melt-quench'

In [321]:
for i in mq_db_G18lab_train:
    i.info['config_type'] = 'melt-quench'

In [318]:
# run_97 etc. didn't finish, so not exactly 80% but pretty close
len(mq_db_G18lab_train)/len(mq_db_G18lab_val)


len(mq_db_G18lab_train),len(mq_db_G18lab_val)

(22810, 5784)

In [ ]:
# with open('mqdb_GAP18labelled/mqdb_GAP18labelled_train.cfg', 'w') as f:
#     write_cfg_db(f, mq_db_G18lab_train, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [ ]:
# with open('mqdb_GAP18labelled/mqdb_GAP18labelled_validate.cfg', 'w') as f:
#     write_cfg_db(f, mq_db_G18lab_val, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [176]:
with open('mqdb_GAP18labelled/mqdb_GAP18labelled_train_stressfix.cfg', 'w') as f:
    write_cfg_db(f, mq_db_G18lab_train, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [175]:
with open('mqdb_GAP18labelled/mqdb_GAP18labelled_validate_stressfix.cfg', 'w') as f:
    write_cfg_db(f, mq_db_G18lab_val, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [ ]:
GAP18_mqdb_train = GAP_18_db + mq_db_G18lab_train

In [ ]:

GAP18_mqdb_train = GAP_18_db + mq_db_G18lab_train

In [328]:
mq_db_G18lab_val[0].info

{'rundir': 'run_8',
 'gap18_energy': -10403.180443322573,
 'gap18_virial': array([[ 2.37589791, -5.57641039, -5.43882383],
        [-5.57641039,  1.49713555,  3.60776824],
        [-5.43882383,  3.60776824, -7.73644752]]),
 'timestep': 171.0,
 'config_type': 'melt-quench'}

In [ ]:
# with open('mqdb_GAP18labelled/GAP18_mqdb_GAP18labelled_train.cfg', 'w') as f:
#     write_cfg_db(f, GAP18_mqdb_train, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')


## Generate GAP_labelled crystal structures from the MP
Free us from dependence on the GAP-18 database

In [ ]:
import pymatgen.ext.matproj as mp
import pymatgen.io.ase as pase
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [ ]:
with mp.MPRester("v0JVgKLlpTXKEhiKeDIC") as m:
    results = m.query(criteria={"pretty_formula":'Si'},
                            properties=["exp.tags", "icsd_ids", "spacegroup",
                                        "task_id", "density", "final_structure",
                                        "e_above_hull", "energy", "energy_per_atom"])

mp_xtals = [pase.AseAtomsAdaptor().get_atoms(
            SpacegroupAnalyzer(i['final_structure']).get_conventional_standard_structure())
            for i in results]

In [ ]:
def get_supercell_factor(at, target_size=64):
    f = np.ceil((target_size/len(at))**(1/3))
    #print('supercell factor: %f leads to %f atoms' % (f, len(at)*f**3))
    return f

In [ ]:
# ncopies = 50
ncopies = 10
supers = [[] for i in results]
for i, val in enumerate(mp_xtals):
    for j in range(ncopies):
        s = build.supercells.make_supercell(
                val, get_supercell_factor(val)*np.identity(3))
        s.rattle(0.01) # randomise the positions slightly
        s.set_cell(s.get_cell_lengths_and_angles() * np.concatenate([normal(1, 0.03, (3)), normal(1, 0.01, (3))]),
                 scale_atoms=True) # randomise lattice vectors
        s.info['config_type'] = results[i]['task_id']
        supers[i].append(s)

In [ ]:
sum([len(i) for i in flatten(supers)]), sum([len(i) for i in mq_db_G18lab_train])

In [ ]:
# need to label with GAP-18 now
with open('mp_mq_db_GAP18labelled/mp_db_train_test.xyz', 'w') as f:
    write_xyz(f, flatten(supers[:5]))

In [ ]:
# with open('mp_mq_db_GAP18labelled/mp_db_train_test.cfg', 'w') as f:
#     write_cfg_db(f, mp_db_GAP18labelled_train[:5], energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [ ]:
with open('mp_mq_db_GAP18labelled/mp_db_validate.xyz', 'w') as f:
    write_xyz(f, flatten(supers))

In [249]:
with open('mp_mq_db_GAP18labelled/mp_db_GAP18labelled_train.xyz', 'r') as f:
    mp_db_GAP18labelled_train = list(read_xyz(f, index=slice(0,None)))

In [293]:
with open('mp_mq_db_GAP18labelled/mp_db_GAP18labelled_validate.xyz', 'r') as f:
    mp_db_GAP18labelled_val = list(read_xyz(f, index=slice(0,None)))

In [87]:
mp_db_GAP18labelled_train[0].calc = GAP_18_pot
mp_db_GAP18labelled_train[0].info, mp_db_GAP18labelled_train[0].get_stress(), GAP_18_pot.extra_results, mp_db_GAP18labelled_train[0].get_volume()


(array([-0.01757834, -0.04315108, -0.01966579, -0.00241345,  0.00664968,
        -0.00039532]),
 {'config': {'virial': array([[17.07531956,  0.38400455, -6.45939087],
          [ 0.38400455, 41.91628217,  2.34438589],
          [-6.45939087,  2.34438589, 19.10303691]])},
  'atoms': {}},
 971.3842829785646)

In [260]:
for i in mp_db_GAP18labelled_train:
    i.info['gap18_energy'] = i.info.pop('energy')
    i.info['gap18_virial'] = i.info.pop('virial')
    i.arrays['gap18_force'] = i.arrays.pop('force')


In [295]:
for i in mp_db_GAP18labelled_val:
    i.info['gap18_energy'] = i.info.pop('energy')
    i.info['gap18_virial'] = i.info.pop('virial')
    i.arrays['gap18_force'] = i.arrays.pop('force')

In [266]:
with open('MP_dbs/mp_db_GAP18labelled_train.cfg', 'w') as f:
    write_cfg_db(f, mp_db_GAP18labelled_train, energy_name='gap18_energy',
                               force_name='gap18_force', virial_name='gap18_virial')

In [267]:
with open('MP_dbs/mp_db_GAP18labelled_validate.cfg', 'w') as f:
    write_cfg_db(f, mp_db_GAP18labelled_val, energy_name='gap18_energy',
                               force_name='gap18_force', virial_name='gap18_virial')

In [268]:
with open('MP_dbs/mp_db_g18_GAP18labelled_train.cfg', 'w') as f:
    write_cfg_db(f, mp_db_GAP18labelled_train + GAP_18_db, energy_name='gap18_energy',
                               force_name='gap18_force', virial_name='gap18_virial')

In [ ]:
mp_mq_db_GAP18labelled_train = mq_db_G18lab_train + mp_db_GAP18labelled_train

# 100k Big melt quench development


In [34]:
#amo_1k = read_cfg('/home/joe/Documents/Ge_od/GAP_to_MTP/pressure_quench_devel/dump_npt_Si_mtp.240000.cfg')
amo_1k_litGAP = read_cfg('/home/joe/Documents/Ge_od/GAP_to_MTP/MTP_mds/GAPmd_run_1000_lit/NPT/dump_npt_Si_gap.240000.cfg')

In [37]:
write_lammps_data('/home/joe/Documents/Ge_od/GAP_to_MTP/pressure_quench/amo_1k_litGAP.data', amo_1k_litGAP)

In [27]:
with open('/home/joe/Documents/Ge_od/GAP_to_MTP/MTP_mds/u16_m1.5_run10k/NPT/dump_npt_Si_mtp.240000.cfg', 'r') as f:
    amo_10k = read_cfg(f)

In [48]:
def label_db(db, pot, pot_name, e_name='energy', f_name='force', v_name='virial'):

    for i, val in enumerate(db):
        pot.calculate(val, properties=['energies', 'forces', 'stress'])
        val.info['{}_{}'.format(pot_name, e_name)] = pot.results['energy']
        val.info['{}_{}'.format(pot_name, v_name)] = pot.extra_results['config']['virial']
        val.arrays['{}_{}'.format(pot_name, f_name)] = pot.results['forces']


In [48]:
with open('PQ_GAP18_GAP18labelled/aSi_100k_relaxed_amorphous_labelled.xyz', 'r') as f:
    amo_100k = list(read_xyz(f))
# label_db(amo_100k, GAP_18_pot, 'gap18')


In [54]:
amo_100k[0].info['gap18_energy'] = amo_100k[0].info['energy']
amo_100k[0].info['gap18_virial'] = amo_100k[0].info['virial']
amo_100k[0].arrays['gap18_force'] = amo_100k[0].arrays['force']

In [55]:
PQ100k_GAP18_db_GAP18labelled = GAP_18_db + amo_100k

In [56]:
with open('PQ_GAP18_GAP18labelled/PQ100k_GAP18_db_GAP18labelled.cfg', 'w') as f:
    write_cfg_db(f, PQ100k_GAP18_db_GAP18labelled,
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')


In [ ]:
write_lammps_data('/home/joe/Documents/Ge_od/GAP_to_MTP/pressure_quench/dump_npt_Si_mtp.240000.data', amo_1k)

In [42]:
write_lammps_data('/home/joe/Documents/Ge_od/GAP_to_MTP/pressure_quench/amo_100k.data', amo_100k)

In [29]:
write_lammps_data('/home/joe/Documents/Ge_od/GAP_to_MTP/pressure_quench/amo_10k.data', amo_10k)

## Write 1k randstart pressure quench to DB for additional fitting

In [92]:
GAP18_PQ = MD_run('/u/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP/GAP-18_ref/GAP18_PQ_1k/',
                  label='GAP18_PQ_1k', read_dat=True)

In [93]:
GAP18_PQ.df['Configs'][1000].info

{'file': '/u/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP/GAP-18_ref/GAP18_PQ_1k/NPT/dump_npt_Si_gap.1000.cfg'}

In [94]:
label_db(GAP18_PQ.df['Configs'].tolist(), GAP_18_pot, 'GAP18')

NameError: name 'label_db' is not defined

In [95]:
GAP_18_pot.calculate(GAP_18_db[200], properties=['energies', 'forces', 'virial'])

In [96]:
GAP_18_pot.extra_results['config']['virial']

array([[ 7.12239936e+01, -3.53353646e-01, -6.43044093e-02],
       [-3.53353646e-01,  6.76416908e+01, -6.86723415e-02],
       [-6.43044093e-02, -6.86723415e-02,  6.37228947e+01]])

In [97]:
GAP_18_pot.results['stress']

array([-8.05836337e-02, -7.65305757e-02, -7.20968054e-02,  7.76966657e-05,
        7.27547376e-05,  3.99788321e-04])

In [102]:
mtp_stress_test.calculate(GAP_18_db[200], properties=['forces', 'stress'])

In [103]:
mtp_stress_test.results['stress']

array([[ 6.066771e+01, -2.156200e-01, -7.920000e-03],
       [-2.156200e-01,  5.668705e+01, -7.620000e-03],
       [-7.920000e-03, -7.620000e-03,  4.340911e+01]])

In [8]:
GAP_18_db[0].info

{'config_type': 'isolated_atom',
 'gap_energy': -157.7272532,
 'gap_virial': array([0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'dft_energy': -158.54496821,
 'cutoff': 5.5,
 'nneightol': 1.2,
 'energy': -158.54496700404255,
 'stress': array([-0., -0., -0., -0., -0., -0.]),
 'gap18_energy': -158.54496700404255,
 'gap18_virial': array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])}

In [65]:
with open('PQ_GAP18labelled_train.cfg', 'w') as f:
    write_cfg_db(f, GAP18_PQ.df['Configs'].tolist(), energy_name='GAP18_energy', force_name='GAP18_force', virial_name='GAP18_virial')

In [ ]:
cfg1 = list(read_cfg_db('GAP_18_db_GAPlabelled.cfg'))

In [9]:
cfg2 = list(read_cfg_db('PQ_GAP18labelled_train.cfg',
                        energy_label='gap18_energy', force_label='gap18_force', stress_label='gap18_virial'))

In [11]:
cfg3 = cfg2 + GAP_18_db

In [14]:
with open('PQ_GAP18_GAP18labelled_train.cfg', 'w') as f:
    write_cfg_db(f, cfg3, 
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

## Write 10k pressure quench to DB

In [5]:
with open('./GAP-18_ref/GAP18_PQ_10k_label_G18.xyz', 'r') as f:
    GAP18_10k_PQ = list(read_xyz(f, index=slice(0,None)))

In [290]:
GAP18_10k_PQ[-1].info

{'file': 'GAP18_PQ_10k/NPT/dump_npt_Si_gap.251000.cfg',
 'timestep': 251000,
 'CPU': 410.93884,
 'Temp': 501.4184,
 'f_TempAve': 501.31106,
 'Press': 201066.87,
 'f_PressAve': 200094.07,
 'f_PEAve_Atom': -162.67298,
 'Volume': 133526.57,
 'f_vAve': 133599.33,
 'c_MSD[4]': 39.187464,
 'GAP18_energy': -1626729.54722908,
 'GAP18_virial': array([-0.12126954, -0.1302307 , -0.11527369, -0.00446968, -0.00630133,
        -0.00535704]),
 'gap18_energy': -1626729.54722908,
 'gap18_virial': array([16192.74069722, 17389.29658431, 15392.13375815,   596.82253473,
          841.39698258,   715.30928784])}

In [247]:
for i in GAP18_10k_PQ:
    i.info['gap18_energy'] = i.info['GAP18_energy']
#     i.info['gap18_virial'] = i.info['GAP18_virial'] # old mistake line
    i.info['gap18_virial'] = -1*i.info['GAP18_virial']*i.get_volume() # key line
    i.arrays['gap18_force'] = i.arrays['GAP18_force']

In [288]:
with open('PQ10k_amo_dbs/PQ10k_amo_G18_GAP18labelled_train_plusstress.cfg', 'w') as f:
    write_cfg_db(f, GAP18_10k_PQ+GAP_18_db, 
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [289]:
with open('PQ10k_amo_dbs/PQ10k_GAP18labelled_train.cfg', 'w') as f:
    write_cfg_db(f, GAP18_10k_PQ, 
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

## 10k-rand compression to DB

In [48]:
os.getcwd()

'/home/tridymite/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP'

In [50]:
with open('./GAP-18_ref/GAP18_PQ_10k_rand_label_G18.xyz', 'r') as f:
    GAP18_10k_PQ_rand = list(read_xyz(f, index=slice(0,None)))

In [245]:
for i in GAP18_10k_PQ_rand:
    i.info['gap18_energy'] = i.info['GAP18_energy']
#     i.info['gap18_virial'] = i.info['GAP18_virial'] # old mistake line
    i.info['gap18_virial'] = -1*i.info['GAP18_virial']*i.get_volume() # key line
    i.arrays['gap18_force'] = i.arrays['GAP18_force']

In [287]:
with open('PQ10k_rand_dbs/PQ10k_rand_G18_GAP18labelled_train_plusstress.cfg', 'w') as f:
    write_cfg_db(f, GAP18_10k_PQ_rand+GAP_18_db, 
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

# 10x1k compression from random start

started fixing the stresses -
'GAP18_virial' is the stress as calculated by quippy.get_stress()

In [236]:
GAP_18_db[500].info['stress'][0:3], -GAP_18_db[500].info['gap18_virial'].diagonal()/GAP_18_db[500].get_volume()

(array([-0.14023119, -0.13998424, -0.1383316 ]),
 array([-0.14023119, -0.13998424, -0.1383316 ]))

In [ ]:
for i in GAP_18_db:
    i.info['gap18_stress'] = -i.info['stress']

In [223]:
MD_1ks_rand_dirs = sorted([i for i in os.listdir('GAP-18_ref/GAP18_1k_PQs_rand') if '.xyz' in i],
                     key=lambda f: int(f.split('_')[2]))
MD_1ks_rand = [MD_run(join('GAP-18_ref/GAP18_1k_PQs_rand',i), label=i.split('_')[2], format='xyz') for i in MD_1ks_rand_dirs]

In [196]:
sum(MD_1ks_rand[0].df['Configs'][251000].info['GAP18_virial'][0:3])/3*-160

19.251663911410873

In [227]:
-MD_1ks_rand[0].df['Configs'][251000].info['GAP18_virial']*MD_1ks_rand[0].df['Configs'][251000].get_volume()

array([1541.99770981, 1570.68062337, 1584.01900447,   -4.74980168,
         10.07783186,   -5.43572035])

In [226]:
MD_1ks_rand[0].df['Configs'][251000].info['gap18_virial']

array([1541.99770981, 1570.68062337, 1584.01900447,   -4.74980168,
         10.07783186,   -5.43572035])

In [225]:
# for j in MD_1ks_rand:
#     for i in j.df['Configs']:
#         i.info['gap18_energy'] = i.info['GAP18_energy']
#         i.info['gap18_virial'] = i.info['GAP18_virial']
#         i.arrays['gap18_force'] = i.arrays['GAP18_force']
for j in MD_1ks_rand:
    for i in j.df['Configs']:
        i.info['gap18_energy'] = i.info['GAP18_energy']
        i.info['gap18_stress'] = -1*i.info['GAP18_virial']
        i.info['gap18_posstress'] = i.info['GAP18_virial']
        i.info['gap18_virial'] = -1*i.info['GAP18_virial']*i.get_volume() # key line
        i.arrays['gap18_force'] = i.arrays['GAP18_force']
        

In [228]:
db10 = []
for i in MD_1ks_rand:
    db10.extend(i.df['Configs'].tolist())

In [229]:
with open('PQ1k_rand_dbs/PQ1k_rand_1-10_G18_GAP18labelled_train_plusstress.cfg', 'w') as f:
    write_cfg_db(f, db10+GAP_18_db, 
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [230]:
db5 = []
for i in MD_1ks_rand[:5]:
    db5.extend(i.df['Configs'].tolist())

In [231]:
with open('PQ1k_rand_dbs/PQ1k_rand_1-5_G18_GAP18labelled_train_plusstress.cfg', 'w') as f:
    write_cfg_db(f, db5+GAP_18_db, 
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [207]:
# with open('PQ1k_rand_1_G18_GAP18labelled_train_stressfix_neg.cfg', 'w') as f:
#     write_cfg_db(f, MD_1ks_rand[0].df['Configs'].tolist() + GAP_18_db, 
#                  energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_stress')

In [212]:
# with open('PQ1k_rand_1_G18_GAP18labelled_train_stressfix_pos.cfg', 'w') as f:
#     write_cfg_db(f, MD_1ks_rand[0].df['Configs'].tolist() + GAP_18_db, 
#                  energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_posstress')

In [213]:
# with open('PQ1k_rand_1_G18_GAP18labelled_train_nostress.cfg', 'w') as f:
#     write_cfg_db(f, MD_1ks_rand[0].df['Configs'].tolist() + GAP_18_db, 
#                  energy_name='gap18_energy', force_name='gap18_force', virial_name='None')

In [180]:
# with open('PQ1k_rand_10_GAP18labelled_train_stressfix_neg.cfg', 'w') as f:
#     write_cfg_db(f, MD_1ks_rand[-1].df['Configs'].tolist(), 
#                  energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_stress')

# Compressions-only 
(200 ps of snapshots, avoid random structure)

In [67]:
db5_comp = []
for i in MD_1ks_rand[:5]:
    db5_comp.extend(i.df['Configs'][i.df.index[51:]].tolist())

In [68]:
with open('PQ1k_rand_1-5_compress_only_G18_GAP18labelled_train.cfg', 'w') as f:
    write_cfg_db(f, db5_comp+GAP_18_db, 
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

# Write 10 x 1k pressure quench to DB, from separate amorphous quenches

In [232]:
MD_1ks_dirs = sorted([i for i in os.listdir('GAP-18_ref/GAP18_1k_PQs') if '.xyz' in i],
                     key=lambda f: int(f.split('_')[2]))
MD_1ks_dirs

['run_1000_1_label_G18.xyz',
 'run_1000_2_label_G18.xyz',
 'run_1000_3_label_G18.xyz',
 'run_1000_4_label_G18.xyz',
 'run_1000_5_label_G18.xyz',
 'run_1000_6_label_G18.xyz',
 'run_1000_7_label_G18.xyz',
 'run_1000_8_label_G18.xyz',
 'run_1000_9_label_G18.xyz',
 'run_1000_10_label_G18.xyz']

In [233]:
MD_1ks = [MD_run(join('GAP-18_ref/GAP18_1k_PQs',i), label=i.split('_')[2], format='xyz') for i in MD_1ks_dirs]

In [234]:
for j in MD_1ks:
    for i in j.df['Configs']:
        i.info['gap18_energy'] = i.info['GAP18_energy']
#         i.info['gap18_virial'] = i.info['GAP18_virial']
        i.info['gap18_virial'] = -1*i.info['GAP18_virial']*i.get_volume()
        i.arrays['gap18_force'] = i.arrays['GAP18_force']
        

In [235]:
MD_1ks[0].df['Configs'][251000].info['gap18_virial']

array([1801.40520798, 1406.95745017, 1695.69978211,   15.02548317,
         88.1204286 ,  -95.26454524])

In [237]:
with open('PQ1k_amo_dbs/PQ1k_1_G18_GAP18labelled_train_amostart_plusstress.cfg', 'w') as f:
    write_cfg_db(f, MD_1ks[0].df['Configs'].tolist()+GAP_18_db, 
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [238]:
db10 = []
for i in MD_1ks:
    db10.extend(i.df['Configs'].tolist())

In [239]:
with open('PQ1k_amo_dbs/PQ1k_1-10_G18_GAP18labelled_train_amostart_plusstress.cfg', 'w') as f:
    write_cfg_db(f, db10+GAP_18_db, 
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [240]:
db5 = []
for i in MD_1ks[:5]:
    db5.extend(i.df['Configs'].tolist())

In [241]:
with open('PQ1k_amo_dbs/PQ1k_1-5_G18_GAP18labelled_train_amostart_plusstress.cfg', 'w') as f:
    write_cfg_db(f, db5+GAP_18_db, 
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [244]:
db5[-1].info['gap18_virial']

array([1679.28314905, 1611.2982686 , 1650.3344295 ,   -5.93457114,
         12.98905522,    6.67528483])

## Writing compressions without crystallisation to DB

In [46]:
db_noncrystallise = []
choice = [2, 5, 6, 8, 9]
for ct, i in enumerate(MD_1ks):
    if ct in choice:
        db_noncrystallise.extend(i.df['Configs'].tolist())

In [47]:
with open('PQ1k_noncryst_G18_GAP18labelled_train_amostart.cfg', 'w') as f:
    write_cfg_db(f, db_noncrystallise+GAP_18_db, 
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [80]:
db_noncrystallise[-10].calc = GAP_18_pot
db_noncrystallise[-10].get_stress()

array([-0.12698578, -0.10429556, -0.11677368, -0.00656289,  0.00558718,
        0.00021673])

## Construct robust testing set for rmses

contains:
1. melt-quench
2. MP_validate
3. amo_compression 10
4. rand_compression 10

In [350]:
rmse_validate = mq_db_G18lab_val + mp_db_GAP18labelled_val + MD_1ks[-1].df['Configs'].tolist()[0:None:10] + MD_1ks_rand[-1].df['Configs'].tolist()[0:None:10]

In [342]:
dbs = [mq_db_G18lab_val[0:None:10] , mp_db_GAP18labelled_val , MD_1ks[-1].df['Configs'].tolist()[0:None:10] , MD_1ks_rand[-1].df['Configs'].tolist()[0:None:10]]

In [343]:
len(mq_db_G18lab_val)

5784

In [344]:
lens = [sum([len(i) for i in j]) for j in dbs]

In [345]:
lens

[47280, 54440, 26000, 26000]

In [336]:
for i in dbs[-1]:
    i.info['config_type'] = 'compress_rand'

In [337]:
for i in dbs[-2]:
    i.info['config_type'] = 'compress_amo'

In [352]:
# with open('MQ_MP_PQ1k_amo_rand_testing.xyz', 'w') as f:
#     write_xyz(f, rmse_validate)